# Training ok

Trained RO-GPT-2-medium on 5 sets of documents: cerere, procura, declaratie, testament and proces-verbal and computed perplexity score for each category.

In [ ]:
!pip install transformers datasets peft accelerate bitsandbytes

In [ ]:
!pip install -U bitsandbytes

In [ ]:
from datasets import load_dataset
from pathlib import Path

dataset_path = "/content/drive/MyDrive/Colab Notebooks/data"

# Load all text files recursively
files = list(Path(dataset_path).rglob("*.txt"))
file_paths = [str(f) for f in files]

dataset = load_dataset("text", data_files=file_paths)
dataset = dataset["train"].train_test_split(test_size=0.2)
print(dataset)


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("readerbench/RoGPT2-medium")
tokenizer.pad_token = tokenizer.eos_token  # GPT2 has no pad token

def tokenize(example):
    return tokenizer(example["text"], truncation=True, max_length=512, padding="max_length")

tokenized_dataset = dataset.map(tokenize, batched=True)


In [ ]:
from transformers import BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True
)


In [ ]:
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType
import torch

model = AutoModelForCausalLM.from_pretrained(
    "readerbench/RoGPT2-medium",
    device_map="auto",       # folosește GPU automat
    dtype=torch.float16,
    offload_folder="offload",
    quantization_config=bnb_config,
)

# LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_proj", "c_attn"],  # typical GPT2 projection layers
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
print(tokenized_dataset["train"][1000])
print(len(tokenized_dataset["train"][0]["input_ids"]))


In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Colab Notebooks/ro-gpt2-legal",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    logging_steps=50,
    save_steps=200,
    eval_steps=200,
    save_total_limit=2,
    learning_rate=2e-4,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    data_collator=data_collator
)


trainer.train()


In [ ]:
model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/ro-gpt2-legal-final")
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/ro-gpt2-legal-final")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Colab Notebooks/ro-gpt2-legal-final")
my_model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/Colab Notebooks/ro-gpt2-legal-final")

def generate_doc(prompt, max_length=500):
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    output = my_model.generate(
        input_ids,
        max_length=max_length,
        do_sample=True,
        temperature=0.2,
        top_p=0.95,
        no_repeat_ngram_size=2,
        pad_token_id=tokenizer.eos_token_id
    )
    text = tokenizer.decode(output[0], skip_special_tokens=True)
    return text

In [ ]:
prompt = """PROCURĂ
Subsemnata, ... (Nume mandantă), cetățean român, cu domiciliul în Mun. București, Sector ..., Str. ..., nr. ..., bl. ..., sc. ..., et. ..., ap. ..., identificată cu C.I. seria ..., nr. ..., eliberată de S.P.C.E.P. Sector ... la data de ..., valabilă până la data de ..., C.N.P. ..., prin prezenta împuternicesc pe doamna ... (Nume mandatară), cetățean român, cu domiciliul în Mun. București, Sector ..., Str. ..., nr. ..., bl. ..., sc. ..., et. ..., ap. ..., posesoare a C.I. seria ..., nr. ..., eliberat de S.P.C.E.P. Sector ... la data de ..., valabil până la data de ..., C.N.P. ..., pentru ca în numele meu şi pentru mine, să acţioneze cu depline puteri, în faţa autorităţilor şi instituţiilor publice, administrative, financiare, bancare, judecătoreşti etc., precum şi în faţa oricăror persoane fizice şi juridice, publice sau private, pentru a îndeplini, încheia şi executa orice acte de administrare şi conservare cu privire la toate bunurile mele mobile şi imobile din România, şi în general, pentru a mă reprezenta, susţine şi apăra drepturile şi interesele mele de orice fel.
În executarea prezentului mandat, mandatara mea mă va putea reprezenta în orice fel de probleme, putând îndeplini orice fel de acte sau fapte, în numele meu şi pentru mine, după cum va considera necesar şi, în îndeplinirea prezentului mandat general, va putea îndeplini următoarele operaţiuni, dar fără a se limita la:
         """
text = generate_doc(prompt)
print(text)


In [ ]:
prompts = {
   "cerere": """
    Domnule Preşedinte,

Subsemnatul_, în calitate de_(reclamant sau, după caz, pârât, intervenient, apelant, recurent, revizuient,
intimat etc.) în dosarul nr. / al acestei instanţe, cu termen de judecată la data de_, vă rog să

""",
    "declaratie": """DECLARAȚIE
Subsemnata ..., cetăţean român, cu domiciliul în municipiul București, str. ... nr. ..., bl. ..., sc. ..., et. ..., ap. ..., sector ..., identificată cu C.I. seria ..., nr. ... eliberată de S.P.C.E.P. Sector ..., la data de ..., C.N.P. ..., în nume propriu, declar următoarele:

""",
    "testament":"""TESTAMENT

          Subsemnatul .......... domiciliat în .........., CNP .........,[1]dispun prin prezentul testament  următoarele:
""",
    "proces_verbal":"""
PROCES-VERBAL DE AUDIERE MINOR

[2]

CU VÂRSTA DE PESTE 10 ANI

          Azi, data de ..........., în faţa mea........, notar public, în cadrul procedurii de
""",
    "procura":"""PROCURĂ SPECIALĂ

Subsemnatul NUME MANDANT, cetățean român, domiciliat în municipiul București, str. ..., nr. ..., bl.3, sc. ..., et. ..., ap. ..., sector 1, identificat cu Carte de Identitate seria ..., nr. ..., eliberată de S.P.C.E.P. Sector 1, la data de ..., C.N.P...., împuternicesc prin prezenta pe domnul NUME MANDATAR, cetățean român, domiciliat în municipiul București, str. ..., nr. ..., bl. ..., sc. ..., et. ..., ap. ..., sector 1, posesor al C.I., seria ..., nr. ..., eliberată de S.P.C.E.P. Sector 1, la data de ..., C.N.P...., ca în numele meu şi pentru mine să se prezinte şi să mă reprezinte, cu puteri depline în vederea îndeplinirii tuturor formalităţilor necesare înmatriculării și înregistrării fiscale și obținerii plăcuțelor cu numere de înmatriculare (pe numele meu ca nou proprietar) privind autoturismul marca ..., număr de identificare ..., an fabricație ..., cu obligația de a face dovada calității mele de nou proprietar.
	În îndeplinirea mandatului de faţă, mandatarul meu este împuternicit, să întocmească
"""
}

In [ ]:
from pathlib import Path

base_dir = Path("/content/drive/MyDrive/Colab Notebooks/generated_docs")
for doc_type,prompt in prompts.items():
    print(f"Generating {doc_type}...")

    for i in range(1, 6):
        text = generate_doc(prompt)
        clean_text = text.replace("\n\n", "\n").strip()
        output_path = base_dir / doc_type / f"{doc_type}_{i}.txt"
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(clean_text)
    print(f"Saved 5 {doc_type} documents in {base_dir/doc_type}")


In [ ]:
from transformers import pipeline
import torch
import math
import numpy as np
from pathlib import Path

device = 0 if torch.cuda.is_available() else -1
model_path = '/content/drive/MyDrive/Colab Notebooks/ro-gpt2-legal-final'

tokenizer = AutoTokenizer.from_pretrained(model_path)

model = AutoModelForCausalLM.from_pretrained(model_path)
if device != -1:
    model.to(f'cuda:{device}')


perplexity_pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=device)

def compute_perplexity(text):
    enc = tokenizer(text, return_tensors="pt")
    enc = {k: v.to(model.device) for k, v in enc.items()}
    with torch.no_grad():
        loss = model(**enc, labels=enc["input_ids"]).loss
    return math.exp(loss.item())

doc_types = ["proces_verbal", "procura", "cerere", "declaratie", "testament"]
base_gen = Path('/content/drive/MyDrive/Colab Notebooks/generated_docs')
for doc_type in doc_types:
    scores = []
    for file in (base_gen / doc_type).glob("*.txt"):
        with open(file, "r", encoding="utf-8") as f:
            txt = f.read().strip()
            if len(txt.split()) > 10:
                try:
                    scores.append(compute_perplexity(txt))
                except:
                    continue
    if scores:
        avg = np.mean(scores)
        print(f"{doc_type.upper()} average perplexity: {avg:.2f}")

# Too long of a training

Tried training RO-LLAMA-2-7B using LoRa, but the training would last 2 months and a half.

In [ ]:
!pip install transformers torch accelerate bitsandbytes


In [ ]:
import torch
print(torch.cuda.is_available())  # True dacă GPU e activ
print(torch.cuda.get_device_name(0))  # Numele GPU-ului


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

In [ ]:
from transformers import BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True
)


In [ ]:
MODEL_NAME = "OpenLLM-Ro/RoLlama2-7b-Base"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",       # folosește GPU automat
    dtype=torch.float16,
    offload_folder="offload",
    quantization_config=bnb_config,

)


In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj","v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
dataset_path = "/content/drive/MyDrive/Colab Notebooks/jurisprudenta.txt"
from datasets import load_dataset

# Dacă fișierul are o propoziție/document per linie
dataset = load_dataset("text", data_files={"train": dataset_path})

print(dataset["train"][0:20])


In [ ]:
def tokenize_fn(example):
    return tokenizer(example["text"], truncation=True, max_length=1024)

tokenized_dataset = dataset.map(tokenize_fn, batched=True, remove_columns=["text"])


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Causal LM, nu Masked LM
)


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./juridical_lora",
    per_device_train_batch_size=1,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    save_strategy="epoch",
    logging_steps=50,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    data_collator=data_collator
)


trainer.train()

model.save_pretrained("./juridical_lora")

# Useful encoder-decoder

Found bert-legal-romanian-cased-v1 model.

In [ ]:
!pip install -U transformers

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="snisioi/bert-legal-romanian-cased-v1")

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("snisioi/bert-legal-romanian-cased-v1")
model = AutoModelForCausalLM.from_pretrained("snisioi/bert-legal-romanian-cased-v1")

In [ ]:
from pathlib import Path
output_dir = Path("/content/drive/MyDrive/Colab Notebooks/generated_docs")
output_dir.mkdir(parents=True, exist_ok=True)


In [ ]:
import os
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_NAME = "readerbench/RoGPT2-medium"
DOC_TYPES = ["cerere", "declaratie", "testament", "proces_verbal"]
OUTPUT_DIR = Path("/content/drive/MyDrive/Colab Notebooks/generated_docs")
NUM_PER_TYPE = 1  # number of docs to generate per type
MAX_LENGTH = 700  # max tokens per generation

PROMPTS = {
    "cerere": "CERERE\nSubsemnata_, domiciliată în_",
    "declaratie": "DECLARAȚIE\nSubsemnata_, cetățean român, cu domiciliul în_",
    "testament": "TESTAMENT\nSubsemnata_, cu domiciliul în_",
    "proces_verbal": "PROCES VERBAL\nSubsemnatul_, reprezentant al_"
}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

In [ ]:
def safe_filename(name):
    # make lowercase, replace spaces with hyphens
    return name.lower().replace(" ", "_") + ".txt"

In [ ]:
for doc_type in DOC_TYPES:
    prompt = PROMPTS[doc_type]
    for i in range(1, NUM_PER_TYPE + 1):
        inputs = tokenizer.encode(prompt, return_tensors="pt")
        generated = model.generate(
            inputs,
            max_length=MAX_LENGTH,
            no_repeat_ngram_size=4,
            do_sample=True,
            top_p=0.9,
            temperature=0.3
        )
        text = tokenizer.decode(generated[0], skip_special_tokens=True)

        file_path = OUTPUT_DIR / doc_type / safe_filename(f"{doc_type}_{i}_temp_03")
        file_path.write_text(text, encoding="utf-8")
        print(f"✅ Generated {file_path}")
